# Statistical API Multiple Output and Data Mask Demonstrator

In [1]:
import os
import yaml
from munch import munchify

### _Setup configuration and environment_

In [2]:
# define repo name and get root working directory
repo = 'statistical-api'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]

# get path to configuration files
cfg_path = os.path.join( root_path, 'cfg' )
cfg_path = os.path.join( cfg_path, 'sentinel-2' )

In [3]:
# get path to configuration files
cfg_path = os.path.join( root_path, 'cfg' )
cfg_path = os.path.join( cfg_path, 'sentinel-2' )
cfg_path

'C:\\Users\\crwil\\Documents\\GitHub\\sentinelhub\\statistical-api\\cfg\\sentinel-2'

In [4]:
# get pathname to configuration file
cfg_file = os.path.join( cfg_path, 's2-multi-output-mask.yml' )

In [5]:
# print contents of configuration file
with open( cfg_file ) as f:
    print ( f.read() )

request:
    evalscript: |       
                //VERSION=3
                function setup() {
                    return {
                        input: [{
                            bands: [
                                "B04",
                                "B08",
                                "SCL",
                                "dataMask"
                            ]
                        }],
                        output: [
                        {
                            id: "output_my_bands",
                            bands: ["only_band_B04", "only_band_B08"],
                            sampleType: "FLOAT32"
                        },
                        {
                            id: "output_my_indices",
                            bands: 1,
                            sampleType: "FLOAT32"
                        },
                        {
                            id: "output_scl",
                            bands: 1,
                      

In [6]:
# load cfg file using yaml parser
with open( cfg_file, 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )
config

Munch({'request': Munch({'evalscript': '//VERSION=3\nfunction setup() {\n    return {\n        input: [{\n            bands: [\n                "B04",\n                "B08",\n                "SCL",\n                "dataMask"\n            ]\n        }],\n        output: [\n        {\n            id: "output_my_bands",\n            bands: ["only_band_B04", "only_band_B08"],\n            sampleType: "FLOAT32"\n        },\n        {\n            id: "output_my_indices",\n            bands: 1,\n            sampleType: "FLOAT32"\n        },\n        {\n            id: "output_scl",\n            bands: 1,\n            sampleType: "UINT8"\n        },\n        {\n            id: "dataMask",\n            bands: ["output_my_bands", "output_my_indices"]\n        }]\n    }\n}\nfunction evaluatePixel(samples) {\n    let ndvi = (samples.B08 - samples.B04)/(samples.B08 + samples.B04)\n    var validNDVIMask = 1\n    if (samples.B08 + samples.B04 == 0 ){\n        validNDVIMask = 0\n    }\n    var noWa

### _Replicate use case results taken from Sentinel-Hub documentation_

In [7]:
# define aggregation timeframe
from datetime import datetime
timeframe = { 'start' : datetime.strptime('2020-07-01', '%Y-%m-%d'), 
              'end' : datetime.strptime('2020-07-15', '%Y-%m-%d') }

In [8]:
# create instance of shclient class
from statisticalapi import Client
client = Client( config )

In [9]:
from sentinelhub import CRS

# define min and max latlons
coords = 414315, 4958219, 414859, 4958819
crs = CRS( 32633 )
bbox = client.getBoundingBox( coords, src_crs=crs )

In [10]:
# submit request
response = client.getStatistics( [ timeframe ], resolution=20, bbox=bbox, interval='P5D' )

In [11]:
response._dfs[ 0 ]

,interval_from,interval_to,output_my_bands_only_band_B04_min,output_my_bands_only_band_B04_max,output_my_bands_only_band_B04_mean,output_my_bands_only_band_B04_stDev,output_my_bands_only_band_B04_sampleCount,output_my_bands_only_band_B04_noDataCount,output_my_bands_only_band_B04_percentiles_33.0,output_my_bands_only_band_B04_percentiles_66.0,...,output_scl_B0_sampleCount,output_scl_B0_noDataCount,output_scl_B0_histogram,output_my_indices_B0_min,output_my_indices_B0_max,output_my_indices_B0_mean,output_my_indices_B0_stDev,output_my_indices_B0_sampleCount,output_my_indices_B0_noDataCount,output_my_indices_B0_histogram
0,2020-07-01,2020-07-06,0.0804,0.2940,0.114511,0.032770,810,0,0.0972,0.1117,...,810,0,"{'bins': [{'lowEdge': 0, 'highEdge': 1, 'count...",-0.040501,0.533831,0.145994,0.156712,810,0,"{'bins': [{'lowEdge': 0.0, 'highEdge': 0.05, '..."
1,2020-07-06,2020-07-11,0.0075,0.3789,0.055661,0.060176,810,0,0.0227,0.0444,...,810,0,"{'bins': [{'lowEdge': 0, 'highEdge': 1, 'count...",-0.189765,0.858506,0.479659,0.251893,810,428,"{'bins': [{'lowEdge': 0.0, 'highEdge': 0.05, '..."


In [12]:
response._dfs[ 0 ].columns

Index(['interval_from', 'interval_to', 'output_my_bands_only_band_B04_min',
       'output_my_bands_only_band_B04_max',
       'output_my_bands_only_band_B04_mean',
       'output_my_bands_only_band_B04_stDev',
       'output_my_bands_only_band_B04_sampleCount',
       'output_my_bands_only_band_B04_noDataCount',
       'output_my_bands_only_band_B04_percentiles_33.0',
       'output_my_bands_only_band_B04_percentiles_66.0',
       'output_my_bands_only_band_B04_percentiles_100.0',
       'output_my_bands_only_band_B08_min',
       'output_my_bands_only_band_B08_max',
       'output_my_bands_only_band_B08_mean',
       'output_my_bands_only_band_B08_stDev',
       'output_my_bands_only_band_B08_sampleCount',
       'output_my_bands_only_band_B08_noDataCount',
       'output_my_bands_only_band_B08_histogram', 'output_scl_B0_min',
       'output_scl_B0_max', 'output_scl_B0_mean', 'output_scl_B0_stDev',
       'output_scl_B0_sampleCount', 'output_scl_B0_noDataCount',
       'output_scl_B0

In [16]:
response._dfs[ 0 ].iloc[ 0 ][ 'output_my_bands_only_band_B08_histogram' ]

{'bins': [{'lowEdge': 0.0, 'highEdge': 0.09999999999999999, 'count': 199},
  {'lowEdge': 0.09999999999999999,
   'highEdge': 0.19999999999999998,
   'count': 270},
  {'lowEdge': 0.19999999999999998, 'highEdge': 0.3, 'count': 332}],
 'overflowCount': 9,
 'underflowCount': 0,
 'normalised_counts': [25, 34, 41],
 'total_counts': 801,
 'bin_edges': [0.0, 0.09999999999999999, 0.19999999999999998, 0.3]}

### _Results extracted from Sentinel-Hub documentation_

'only_band_B08': {'stats': {'min': 0.08410000056028366,
        'max': 0.3700999915599823,
        'mean': 0.16619444458205035,
        'stDev': 0.07349787029694223,
        'sampleCount': 810,
        'noDataCount': 0},
       'histogram': {'bins': [{'lowEdge': 0.0,
          'highEdge': 0.09999999999999999,
          'count': 197},
         {'lowEdge': 0.09999999999999999,
          'highEdge': 0.19999999999999998,
          'count': 275},
         {'lowEdge': 0.19999999999999998, 'highEdge': 0.3, 'count': 313}],
        'overflowCount': 0,
        'underflowCount': 25}}}},

In [14]:
response._dfs[ 0 ].iloc[ 0 ][ 'output_scl_B0_histogram' ]

{'bins': [{'lowEdge': 0, 'highEdge': 1, 'count': 0},
  {'lowEdge': 1, 'highEdge': 2, 'count': 0},
  {'lowEdge': 2, 'highEdge': 3, 'count': 0},
  {'lowEdge': 3, 'highEdge': 4, 'count': 0},
  {'lowEdge': 4, 'highEdge': 5, 'count': 0},
  {'lowEdge': 5, 'highEdge': 6, 'count': 0},
  {'lowEdge': 6, 'highEdge': 7, 'count': 0},
  {'lowEdge': 7, 'highEdge': 8, 'count': 0},
  {'lowEdge': 8, 'highEdge': 9, 'count': 99},
  {'lowEdge': 9, 'highEdge': 10, 'count': 1},
  {'lowEdge': 10, 'highEdge': 11, 'count': 710}],
 'overflowCount': 0,
 'underflowCount': 0,
 'normalised_counts': [0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 88],
 'total_counts': 810,
 'bin_edges': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}

### _Results extracted from Sentinel-Hub documentation_

'output_scl': {'bands': {'B0': {'stats': {'min': 8.0, 'max': 10.0, 'mean': 9.75432098765432, 'stDev': 0.6555648554361158, 'sampleCount': 810, 'noDataCount': 0}, 'histogram': {'bins': [{'lowEdge': 0, 'highEdge': 1, 'count': 0}, {'lowEdge': 1, 'highEdge': 2, 'count': 0}, {'lowEdge': 2, 'highEdge': 3, 'count': 0}, {'lowEdge': 3, 'highEdge': 4, 'count': 0}, {'lowEdge': 4, 'highEdge': 5, 'count': 0}, {'lowEdge': 5, 'highEdge': 6, 'count': 0}, {'lowEdge': 6, 'highEdge': 7, 'count': 0}, {'lowEdge': 7, 'highEdge': 8, 'count': 0}, {'lowEdge': 8, 'highEdge': 9, 'count': 99}, {'lowEdge': 9, 'highEdge': 10, 'count': 1}, {'lowEdge': 10, 'highEdge': 11, 'count': 710}], 'overflowCount': 0, 'underflowCount': 0}}}}